In [6]:
import csv
import json
import os
import requests
import pandas as pd
from dotenv import load_dotenv; load_dotenv()

Loading json file containing hotel data

In [2]:
json_file = "Kuala_Lumpur.json"  # to edit
filepath = "/Users/pritish/projects/Hotels-Decision-Support-and-Sentimental-Analysis" #to edit

with open(os.path.join(filepath,json_file)) as file:
    data = json.load(file)

Requesting geocode data through mapbox using hotel names.

In [ ]:
token = os.getenv('mapbox_token')

for name in data.keys():
    search_text = name
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{search_text}.json"
    response = requests.get(url, params = {'access_token':token}).json()
    # accessing coordinates
    features = response['features']
    geometry = features[0]['geometry']
    coordinates = geometry['coordinates']
    # Add geocode data to hotel data
    data[name]['geocode'] = coordinates
    print(name)

Adding co-ordinates to JSON data files

In [ ]:
with open(os.path.join(filepath,json_file), 'w') as outfile:
            json.dump(data, outfile, indent=4)

Extracting hotel names and geocodes into a Data Frame

In [3]:
to_add = []
columns = ['Hotel_Name','Longitude','Latitude']
for i in data.keys():
    values = [i, data[i]['geocode'][0], data[i]['geocode'][1]]
    zipped_dict = dict(zip(columns, values))
    to_add.append(zipped_dict)
# Making pandas dataframe
df = pd.DataFrame(to_add)
df.head()

,Hotel_Name,Longitude,Latitude
0,Element Kuala Lumpur Hotel,101.685774,3.135573
1,"The RuMa Hotel and Residences, Kuala Lumpur",101.714528,3.152809
2,Four Seasons Hotel Kuala Lumpur,101.713893,3.158010
3,"Mandarin Oriental, Kuala Lumpur Hotel",101.711847,3.156755
4,Banyan Tree Kuala Lumpur Hotel,101.665405,3.170819


Write geocode pertinent data to a csv file


In [4]:
output_filename = json_file[:-5]
with open(os.path.join(filepath,f'{output_filename}_hotelsGeocode.csv'), 'w', newline='') as file:
    df.to_csv(file)